In [44]:
import numpy as np
import pandas as pd
import nltk
from nltk import sent_tokenize,word_tokenize 
import os
import re

import spacy

In [45]:
directory = './CFPB'

### First Format: Company time right after an address

In [71]:
f = open('./CFPB/CFPB-2011-0008/Proposed Rule/CFPB-2011-0008-0001/comment/CFPB-2011-0008-0300_attachment_1.txt', "r", encoding="ISO-8859-1")
text = f.read()
text

" \nMarch 16, 2012 \nThe Honorable Richard Cordray \nDirector \nConsumer Financial Protection Bureau 1500 Pennsylvania Ave. NW (Attn: 1801 L St.) Washington DC 20220 \n \nDear Director Cordray: \nThe National Community Reinvestment Coalition (NCRC) believes that the Consumer Financial Protection Bureau (CFPB) has a critical opportunity to create a safe and sound mortgage marketplace through its promulgation of a rule implementing the qualified mortgage (QM) provision of the Dodd-Frank Wall Street Reform and Consumer Protection Act of 2010. We understand that the Center for Responsible Lending, the Consumer Federation of America, the Leadership Conference on Civil Rights, and the Clearing House Association have submitted a comment letter (joint comment) that proposes a standard for Qualified Mortgages (QM). The joint comment letter represents important advances in creating a consensus for a solid QM rule, but it still would leave consumers without adequate protections against unscrupulo

In [72]:
nlp = spacy.load("en_core_web_sm")

In [73]:
def find_by_spacy(text):
    res = ''
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["ORG"]:
            if (('Reg' not in ent.text) 
            or ('Bureau of Consumer Financial Protection' not in ent.text) 
            or ('Consumer Financial Protection Bureau' not in ent.text)) :
                res += ent.text
    return res

In [59]:
def find_by_address(text):
    regexp = "[0-9]{1,5} .+, .+ [0-9]{5}"
    address = re.findall(regexp, text)

    try:
        for i in range(len(address)):
            if len(address[i]) > 50:
                address.pop(i)
                
        regex = re.compile('\n{0,1}(.+)\n{0,1}'+address[0])
        res = regex.search(text).group(1) 

        if (('Bureau of Consumer Financial Protection' in res) 
            or ('Washington' in res) 
            or ('P. O. Box' in res)
            or ('Suite' in res) 
            or ('Docket ID' in res)
            or ('Consumer Financial Protection Bureau' in res) 
            or ('P.O. Box' in res) or ('Consumer Financial Protection Bureau' in res)):
            res = ''
    except:
        res = ''
    
    return res

### Second Format: Company time at the very beginning of the letter content

In [60]:
def find_by_opening_line(text):
    try:
        regex = re.compile('On behalf of (.+), ')
        res = regex.search(text).group(1)
    except:
        try:
            regex = re.compile('On behalf of (.+), ')
            res = regex.search(text).group(1)
        except:
            try:
                regex = re.compile('\n+.+\n+(.+)appreciate')
                res = regex.search(text).group(1)
            except:
                try:
                    regex = re.compile('\n.+\n(.+)respectfully submit')
                    res = regex.search(text).group(1)
                except:
                    try:
                        regex = re.compile('\n.+\n(.+)is')
                        res = regex.search(text).group(1)
                    except:
                        res = ''
    
    if len(res) > 40:
        res = ''
    if len(res) <=2:
        res = ''
            
    if (('We ' in res) or ('I ' in res) or ('we ' in res) or (' Reg' in res) or ('Subject:' in res) or ('name ' in res) or (('My ' in res))):
        res = ''
    return res

In [61]:
# 'Respectfully submitted', 'Yours faithfully', 'Sincerely'

In [62]:
# regex = re.compile('Sincerely.+\n((.|\n)+)')
# res = regex.search(text).group(1) 

In [63]:
# res.split('\n')

In [64]:
res_dict = {}
for entry in os.scandir(directory):
    if (entry.path.endswith('.DS_Store')):
        continue
            
    print(entry.path)
    if not entry.path.endswith('.DS_Store'):
        for subentry in os.scandir(entry.path):
            if (subentry.path.endswith("Links.csv") or 
                subentry.path.endswith("Links.txt") or 
                subentry.path.endswith('.DS_Store')):
                continue
       
            if subentry.path.endswith('Proposed Rule'):
                    
                for sub in os.scandir(subentry.path):
                    if (sub.path.endswith("Links.csv") or 
                        sub.path.endswith("Links.txt") or 
                        sub.path.endswith('.DS_Store')):
                        continue
                    docket_id = sub.path.split('/')[2]
                    file_type = sub.path.split('/')[3]
                    pro_rule_id = sub.path.split('/')[4]

                    if docket_id not in res_dict:
                        res_dict[docket_id] = []
#                     if file_type not in res_dict[docket_id]:
#                         res_dict[docket_id][file_type] = {}
#                     if pro_rule_id not in res_dict[docket_id][file_type]:
#                         res_dict[docket_id][file_type][pro_rule_id] = {}

                    for pro_rule_entry in os.scandir(sub.path):
                        if (pro_rule_entry.path.endswith("Links.csv") or 
                        pro_rule_entry.path.endswith("Links.txt") or 
                        pro_rule_entry.path.endswith('.DS_Store')):
                            continue
                        
                        
                        
                        if pro_rule_entry.path.endswith('comment'):
#                             if 'comment' not in res_dict[docket_id][file_type]:
#                                 res_dict[docket_id][file_type][pro_rule_id]['comment'] = []

                            for filename_path in os.scandir(pro_rule_entry.path):
                                filename = filename_path.path
                                if filename.endswith(".txt") and not filename.endswith("Links.txt"):
                                    f = open(filename, "r", encoding="ISO-8859-1")
                                    text = f.read()
#                                     name = find_by_address(text)
#                                     name = find_by_opening_line(text)
                                    name = find_by_spacy(text)
                                    res_dict[docket_id].append(name)
                    else:
                        continue
                            


./CFPB/CFPB-2012-0039
./CFPB/CFPB-2012-0037
./CFPB/CFPB-2012-0031
./CFPB/CFPB-2013-0009
./CFPB/CFPB-2014-0024
./CFPB/OCC-2012-0013
./CFPB/CFPB-2017-0021
./CFPB/CFPB-2014-0031
./CFPB/CFPB-2014-0009
./CFPB/CFPB-2013-0013
./CFPB/HUD-2013-0093
./CFPB/RHS-18-SFH-0020
./CFPB/CFPB-2012-0022
./CFPB/CFPB-2015-0006
./CFPB/CFPB-2012-0046
./CFPB/CFPB-2012-0015
./CFPB/OCC-2013-0009
./CFPB/CFPB-2012-0023
./CFPB/CFPB-2016-0038
./CFPB/CFPB-2012-0005
./CFPB/CFPB-2011-0023
./CFPB/CFPB-2012-0033
./CFPB/CFPB-2012-0034
./CFPB/OCC-2014-0002
./CFPB/CFPB-2012-0050
./CFPB/CFPB-2012-0032
./CFPB/CFPB-2015-0029
./CFPB/CFPB-2017-0008
./CFPB/CFPB-2017-0030
./CFPB/VA-2015-VBA-0004
./CFPB/CFPB-2017-0009
./CFPB/CFPB-2014-0010
./CFPB/CFPB-2013-0005
./CFPB/CFPB-2013-0002
./CFPB/CFPB-2014-0028
./CFPB/CFPB-2013-0020
./CFPB/CFPB-2013-0018
./CFPB/CFPB-2014-0003
./CFPB/OCC-2015-0021
./CFPB/CFPB-2017-0015
./CFPB/CFPB-2013-0010
./CFPB/CFPB-2014-0033
./CFPB/CFPB-2012-0021
./CFPB/CFPB-2011-0009
./CFPB/CFPB-2012-0010
./CFPB/CFPB-

In [66]:
# res_dict

In [67]:
d = res_dict

In [68]:
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [69]:
df

,CFPB-2012-0039,CFPB-2012-0037,CFPB-2012-0031,CFPB-2013-0009,CFPB-2014-0024,OCC-2012-0013,CFPB-2017-0021,CFPB-2014-0031,CFPB-2014-0009,CFPB-2013-0013,...,OCC-2015-0021,CFPB-2017-0015,CFPB-2013-0010,CFPB-2014-0033,CFPB-2012-0021,CFPB-2011-0009,CFPB-2012-0010,CFPB-2015-0004,OCC-2017-0008,CFPB-2011-0008
0,The Consumer Financial Protection BureauEvolut...,,Rishel Consulting Groupandtothelandleasecommun...,Monica Jackson OfficeConsumer Financial Protec...,Consumer Financial Protection Bureauthe Automo...,Office of the Comptroller of the Currency Att...,,the Electronic Fund \nTransfer Act (Banking U...,Office of the ExecutiveConsumer Financial Prot...,,...,RIORPOPEJR@TEXASBANKANDTRUST.COM TEXAS BANKTRU...,The Bancorp Bank CommentsPayments Counsel,,Consumer Financial Protection BureauThe Consum...,Consumer Financial Protection Bureau \nRule &...,,,Monica Jackson Office of the ExecutiveConsumer...,RINFDICThe National Community Reinvestment Coa...,
1,Monica Jackson OfficeConsumer Financial Protec...,,Bureau of Consumer Financial ProtectionThe Boa...,ag bankagMN,,,IowaCreditUnions.com \n\nOffice of the Executi...,QtnngresaTh!CConsumer Financial Protection Bur...,Consumer Financial Protection BureauFedEmpire ...,Monica Jackson OfficeConsumer Financial Protec...,...,Washington DCLegislative and Regulatory Divisi...,The American Payroll Associationthe Consumer F...,,Monica Jackson OfficeConsumer Financial Protec...,Consumer Financial ProtectionCFPB.2012.0021 & ...,NAVYCredit UnionOffice of the PresidentBoard o...,,I~.LMID-SIZE BAI'\JK COALITION OF AMERICAConsu...,Office of the Comptroller of the CurrencyThe C...,Luigi Gallegos \nReg Z - Mortgage Repayment S...
2,,Monica Jackson Office of the ExecutiveConsumer...,,Office of the ExecutiveBureau of Consumer Fina...,Hill Country Consumer Law626.737.6030 \n\nOffi...,Comptroller of the CurrencyTreasury Department...,CREDIT UNIONOffice of the ExecutiveConsumer Fi...,Federal Register Citationthe Truth in Lending ...,The Peoples Bank CoColdwaterSmall Creditorthe ...,Monica Jackson OfficeConsumer Financial Protec...,...,Office of the Comptroller of the CurrencyOCCBo...,Consumer Financial Protection BureauConsumer F...,Consumer Financial Protection Bureauthe Real E...,Consumer Financial Protection BureauTILAÒQuick...,etworkBrandedPrepaid Card Associationthe Execu...,,,CooperativeCredit Union AssociationCreating Co...,Office of the Comptroller of the Currencythe F...,Reg Z - Mortgage Repayment StandardsReg
3,Office of the ExecutiveBureau of Consumer Fina...,Bureau of Consumer Financial Protection \nsig...,the National Association of Federal Credit Uni...,CFPB-2013-0009the Bureau \n1SUMMARYBureauNCUA...,,,CFPBCFPBCFPBHMDA,Consumer Financial Protection Bureauthe Consum...,the Center for Responsible Lendingthe Consumer...,,...,NaN,Consumer Financial Protection BureauCFPBÐ2017Ð...,,Consumer Financial Protection BureauRINCommonw...,Consumer Financial Protection Bureauthe Credit...,The Bank of TescottThe Bank of Tescottthe Fede...,Consumer Financial Protection BureauOffice of ...,Consumer Financial Protection BureauThe Connec...,NaN,the Dodd-Frank ActDFATILACongressthe Òqualifie...
4,Bureau of Consumer Financial ProtectionThe Lea...,the Execuve Secretary Bureau of Consumer Finan...,,Office of the ExecutiveConsumer Financial Prot...,,,the Independent Bankers Association of TexasÐ ...,the Truth in Lending Act (Regulation ZCFPB Hea...,,,...,NaN,Federal Register CitationCFPBICUL Service Corp...,,CFPB-2013-0031the National Association of Fede...,Consumer Financial Protection BureauThe Consum...,United Nations Federal Credit UnionVia Email \...,,the \nCenter for Responsible LendingNational ...,NaN,Reg Z - Mortgage Repayment StandardsReg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joe McGuire \nReg Z - Mortgage Repayment Stan...
220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,

In [70]:
df.to_excel("search_spacy_CFPB.xlsx")  

In [32]:
import spacy

In [74]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

March 16, 2012 2 16 DATE
The Honorable Richard Cordray 18 47 WORK_OF_ART
Consumer Financial Protection Bureau 59 95 ORG
1500 96 100 CARDINAL
Pennsylvania 101 113 GPE
1801 129 133 CARDINAL
Washington DC 141 154 GPE
20220 155 160 CARDINAL
Cordray 178 185 PERSON
The National Community Reinvestment Coalition (NCRC 188 239 ORG
the Consumer Financial Protection Bureau 255 295 ORG
CFPB 297 301 ORG
the Dodd-Frank Wall Street Reform and Consumer Protection Act 467 528 ORG
2010 532 536 DATE
the Center for Responsible Lending 557 591 ORG
the Consumer Federation of America 593 627 ORG
the Clearing House Association 676 706 ORG
QM 901 903 PRODUCT
NCRC 1005 1009 ORG
more than 600 1031 1044 CARDINAL
America 1237 1244 GPE
NCRC 1275 1279 ORG
the last several years 1375 1397 DATE
American 1588 1596 NORP
QM 1690 1692 WORK_OF_ART
decades 1891 1898 DATE
CFPB 1937 1941 ORG
the Federal Reserve 2058 2077 ORG
last summer 2106 2117 DATE
CFPB 2150 2154 ORG
CFPB 2324 2328 ORG
the Federal Reserve 2349 2368 ORG
NCR

In [76]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for ent in doc.ents:
    if ent.label_ in ["ORG"]:
        print(ent.text)

Consumer Financial Protection Bureau
The National Community Reinvestment Coalition (NCRC
the Consumer Financial Protection Bureau
CFPB
the Dodd-Frank Wall Street Reform and Consumer Protection Act
the Center for Responsible Lending
the Consumer Federation of America
the Clearing House Association
NCRC
NCRC
CFPB
the Federal Reserve
CFPB
CFPB
the Federal Reserve
NCRC
CFPB
CFPB
CFPB
CFPB
CFPB
CFPB
CFPB
CFPB
CFPB
QM
CFPB
CFPB
QM
NCRC
QM
industryÕs
QM
TDTI
NCRC
HUD
TDTI
NCRC
NCRC
TDTI
QM
CFPB
TDTI
HUD
CFPB
VeteranÕs Affairs
NCRC
CFPB
TDTI
NCRCÕs
the Urban Institute
CFPB
CFPB
CFPB
Federal Reserve
Dodd-Frank
TDTI
CFPB
QM
Research and Policy


In [36]:
doc.ents

(1417,
 Reg Z - Mortgage Repayment Standards,
 Reg,
 Barry Chadwick,
 the day to day)